# Quantifitcation
### A Parent Class that is inherited by DistributionQuantifcation and TransmissionQuantifcation
- Has methods that calculate endstates
    - get_fire_no_injury
    - get_destructive_fire
    - get_small_fire_prob
    - get_safe_prob
    - get_catastrophic_fire
    - get_ac_greater_than_100_fire_endstate
    - get_ac_greater_than_300_fire_endstate
- Also has methods that calculate risk curves and probability histogranms and consequences
- These method are shared by the Distribution and the Transmission Systems, although they may be overwritten to support their differences
- Takes keyword arguments for the following fields, by setting these kw to numbers they are prioritized and considered constants throughout the calculation process
    - psps_not_performed
    - psps
    - ignition1
    - ignition2
    - ignition3
    - outage1
    - outage2
    - catastrophe_ac_threshold
    - catastrophe_bu_threshold
    - catastrophe_pop_threshold
-  .run() method is what calculates everything

# DistributionQuantifcation
- .run() method steps
1. read_initial_settings()
    - reads the settings for the calculation
2. calculate_endstate()
    - will iterate through all of the rows and calculates the endstates based on the parameters per line
3. get_distribution_level_endstate()
    - will calculate endstates at the distribution level using from the line/cpz level
4. get_prob_hist_and_curves()
    - using the endstate values at the cpz level and calculate risk curvers and probability histograms at cpz level
5. calculate_ac_bu_pop_and_risk_totals()
    - calculates ac, bu, pop, and risk curve totals
6. calculate_total_fires()
    - calculates total fires

# TransmissionQuantifcation
- .run() method steps
1. read_initial_settings()
    - reads the settings for the calculation
2. calculate_endstate()
    - will iterate through all of the rows and calculates the endstates based on the parameters per line
3. get_line_level_reduced()
    - will aggregate probability histograms from the Tree and Structure level to the transmission level
4. calculate_ac_bu_pop_and_risk_totals()
    - calculates AC, BU, POP, Financial, Safety, and Reliability Totals at the Tree and Structure level
5. get_line_level_risk_curves()
    - calculates risk curves at the line level from the aggregatino at the 3rd step
6. get_transmission_level_endstate()
    - will calculate transmission level endstates using the endstate values from the Tree and Structure level
7. calculate_total_fires
    - calculates total fires using quantification results at the Tree and Structure level
8. calculate_transmission_line_level_endstates()
    - aggregates Structures and Trees endstates to the Transmission Line Level
9. self.calculate_total_fires_from_lines()
    - Using the aggretation at the line level, calculates total fires

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from collections import defaultdict
import json
from ast import literal_eval
import datetime

In [2]:
distribution = pd.read_csv('../../Distribution/master csv/distribution_with_rc6_drivers.csv')
structures = pd.read_csv('../../Transmission/write and read/structure_complete.csv')
vegetation = pd.read_csv('../../Transmission/write and read/vegetation_complete.csv')

<ipython-input-2-773be03f4965>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  structures = pd.read_csv('../../Transmission/write and read/structure_complete.csv')


In [3]:
class Quantification:
    def __init__(self, **kwargs):
        self.psps_not_performed=kwargs.get('psps_not_performed', 0.05)
        self.psps=kwargs.get('psps', None)
        self.ignition1=kwargs.get('ignition1', 1)
        self.ignition2=kwargs.get('ignition2', 0) 
        self.ignition3=kwargs.get('ignition3', 1)
        self.outage1=kwargs.get('outage1', None)
        self.outage2=kwargs.get('outage2', None)
        self.catastrophe_ac_threshold=kwargs.get('catastrophe_ac_threshold', 300)
        self.catastrophe_bu_threshold=kwargs.get('catastrophe_bu_threshold', 50)
        self.catastrophe_pop_threshold=kwargs.get('catastrophe_pop_threshold', 1)
        # Totals
        self.ac_total = None
        self.pop_total = None
        self.bu_total = None
        self.financial_total = None
        self.reliability_total = None
        self.safety_total = None
        
        
    def log_progress(self, sequence, every=None, size=None, name='Items'):
        from ipywidgets import IntProgress, HTML, VBox
        from IPython.display import display

        is_iterator = False
        if size is None:
            try:
                size = len(sequence)
            except TypeError:
                is_iterator = True
        if size is not None:
            if every is None:
                if size <= 200:
                    every = 1
                else:
                    every = int(size / 200)     # every 0.5%
        else:
            assert every is not None, 'sequence is iterator, set every'

        if is_iterator:
            progress = IntProgress(min=0, max=1, value=1)
            progress.bar_style = 'info'
        else:
            progress = IntProgress(min=0, max=size, value=0)
        label = HTML()
        box = VBox(children=[label, progress])
        display(box)

        index = 0
        try:
            for index, record in enumerate(sequence, 1):
                if index == 1 or index % every == 0:
                    if is_iterator:
                        label.value = '{name}: {index} / ?'.format(
                            name=name,
                            index=index
                        )
                    else:
                        progress.value = index
                        label.value = u'{name}: {index} / {size}'.format(
                            name=name,
                            index=index,
                            size=size
                        )
                yield record
        except:
            progress.bar_style = 'danger'
            raise
        else:
            progress.bar_style = 'success'
            progress.value = index
            label.value = "{name}: {index}".format(
                name=name,
                index=str(index or '?')
            )
        
        
    def get_outage(self, structure_prob, veg_prob):
        return structure_prob + veg_prob - (structure_prob * veg_prob)
    
    """
    Calculates probability histograms by multiplying the each probability of a given histogram
    and with a fire probability, also stores new probabilities in a list based on common values in grouped_hist if not None
    """
    def get_prob_hist(self, prob, hist, grouped_hist=None):
        ac_bu_pop_hist = {'AC':{}, 'BU':{}, 'POP':{}}
        for value, probability in hist['AC']:
            product = prob * probability
            ac_bu_pop_hist['AC'][value]=(product)
            if grouped_hist:
                grouped_hist['AC'][value].append(product)
        
        for value, probability in hist['BU']:
            product = prob * probability
            ac_bu_pop_hist['BU'][value]=(product)
            if grouped_hist:
                grouped_hist['BU'][value].append(product)
            
        for value, probability in hist['POP']:
            product = prob * probability
            ac_bu_pop_hist['POP'][value]=(product)
            if grouped_hist:
                grouped_hist['POP'][value].append(product)
        return ac_bu_pop_hist
        
        
    def get_catastrophic_fire(
        self, high_consequence, 
        psps,ineffective_evacuation, 
        outage_1, outage_2, epss 
        ):

        ineffective_evacuation1 = ineffective_evacuation
        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1 - self.psps_not_performed)*self.ignition2
        value3 = (1-psps) * epss * outage_2 * self.ignition3
        return (value1 + value2 + value3) * high_consequence * ineffective_evacuation1
    
    
    def get_destructive_fire(
        self, high_consequence, 
        psps, ineffective_evacuation, 
        outage_1, outage_2, epss
        ):
 
        ineffective_evacuation1 = ineffective_evacuation
        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1-self.psps_not_performed) * self.ignition2
        value3 = (1-psps) * epss * outage_2 * self.ignition3
        total = (value1 + value2 + value3) * high_consequence * (1-ineffective_evacuation1)

        return total
    
    
    def get_small_fire_prob(
        self, high_consequence, low_consequence, 
        psps, ineffective_evacuation, 
        outage_1, outage_2, epss
        ):
        
        ineffective_evacuation2 = ineffective_evacuation
        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1 - self.psps_not_performed) * self.ignition2
        value3 = (1 - psps) * epss * outage_2 * self.ignition3
        return (value1 + value2 + value3) * (1 - high_consequence) * low_consequence * ineffective_evacuation2
    
    def get_ac_greater_than_100_fire_endstate(self, psps, outage_1, outage_2, ac_greater_than_100, epss):

        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1 - self.psps_not_performed) * self.ignition2
        value3 = (1 - psps) * epss * outage_2 * self.ignition3

        return (value1 + value2 + value3) * ac_greater_than_100

    def get_ac_greater_than_300_fire_endstate(self, psps, outage_1, outage_2, ac_greater_than_300, epss):

        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1 - self.psps_not_performed) * self.ignition2
        value3 = (1 - psps) * epss * outage_2 * self.ignition3

        return (value1 + value2 + value3) * ac_greater_than_300

    
    def get_safe_prob(
        self, high_consequence, 
        low_consequence, psps,
        outage_1, outage_2, epss
        ):
        
        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1 - self.psps_not_performed) * self.ignition2
        value3 = (1 - psps) * epss * outage_2 * self.ignition3
        value4 = (value1 + value2 + value3) * (1 - high_consequence) * (1 - low_consequence)

        value5 = psps * self.psps_not_performed * epss * outage_1 * (1 - self.ignition1)
        value6 = psps * self.psps_not_performed * (1 - (epss * outage_1))
        value7 = psps * (1 - self.psps_not_performed) * (1-self.ignition2)
        value8 = (1 - psps) * epss * outage_2 * (1-self.ignition3)
        value9 = (1 - psps) * (1 - (epss * outage_2))
        
        return value4 + value5 + value6 + value7 + value8 + value9  
    
    
    def get_fire_no_injury(
        self, high_consequence, 
        low_consequence, psps, 
        ineffective_evacuation, 
        outage_1, outage_2, epss
        ):

        ineffective_evacuation2 = ineffective_evacuation
        value1 = psps * self.psps_not_performed * epss * outage_1 * self.ignition1
        value2 = psps * (1 - self.psps_not_performed) * self.ignition2
        value3 = (1 - psps) * epss * outage_2 * self.ignition3
        return (value1 + value2 + value3) * (1 - high_consequence) * low_consequence * (1 - ineffective_evacuation2)

    
    def get_quanitification(self, high_consequence, psps, ineffective_evacuation, outage_1, outage_2, low_consequence, ac_greater_than_100, ac_greater_than_300, epss):
        quantification = {}
        quantification['catastrophic_fire'] = self.get_catastrophic_fire(high_consequence, psps, ineffective_evacuation, outage_1, outage_2, epss)
        quantification['destructive_fire'] = self.get_destructive_fire(high_consequence, psps, ineffective_evacuation, outage_1, outage_2, epss)
        quantification['fire_probability'] = self.get_small_fire_prob(high_consequence, low_consequence, psps, ineffective_evacuation, outage_1, outage_2, epss)
        quantification['fire_probability_no_injury'] = self.get_fire_no_injury(high_consequence, low_consequence, psps, ineffective_evacuation, outage_1, outage_2, epss)
        quantification['safe_probability'] = self.get_safe_prob(high_consequence, low_consequence, psps, outage_1, outage_2, epss)
        quantification['ac_greater_than_100_fire'] = self.get_ac_greater_than_100_fire_endstate(psps, outage_1, outage_2, ac_greater_than_100, epss)
        quantification['ac_greater_than_300_fire'] = self.get_ac_greater_than_300_fire_endstate(psps, outage_1, outage_2, ac_greater_than_300, epss)
        return quantification
    
    
    def get_consequences(self, ac_bu_pop, grouped_hist=None):
        """
        Calculates consequences, and ineffective evacuation
        grouped_hist is used for transmission level
        """
        high_consequence_sums = {'AC':0, 'BU':0}
        low_consequence_sums = {'AC':0, 'BU':0, 'POP':0}
        ineffective_evacuation = 0
        ac_greater_than_100 = 0
        ac_greater_than_300 = 0
        if 'AC' in ac_bu_pop:
            for hist in ac_bu_pop['AC']:
                if hist[1] is not None and hist[0] is not None:
                    if hist[0] == 0:
                        low_consequence_sums['AC'] += hist[1]/19
                    if hist[0] > 300:
                        high_consequence_sums['AC'] += hist[1]/19
                    if hist[0] > 100:
                        ac_greater_than_100 += hist[1]/19
                    if hist[0]> 300:
                        ac_greater_than_300 += hist[1]/19
                    if grouped_hist:
                        grouped_hist['AC'][hist[0]].append(hist[1]/19)

            for hist in ac_bu_pop['BU']:
                if hist[1] is not None and hist[0] is not None:
                    if hist[0] == 0:
                        low_consequence_sums['BU'] += hist[1]/19
                    if hist[0] > 50:
                        high_consequence_sums['BU'] += hist[1]/19
                    if grouped_hist:
                        grouped_hist['BU'][hist[0]].append(hist[1]/19)

            for hist in ac_bu_pop['POP']:
                if hist[1] is not None and hist[0] is not None:
                    if hist[0] == 0:
                        low_consequence_sums['POP'] += hist[1]/19
                    if hist[0]/2000 > 1:
                        ineffective_evacuation += hist[1]/19
                    if grouped_hist:
                        grouped_hist['POP'][hist[0]].append(hist[1]/19)

        high_consequence = high_consequence_sums['AC'] * high_consequence_sums['BU']
        low_consequence = 1 - (low_consequence_sums['AC'] * low_consequence_sums['BU'] * low_consequence_sums['POP'])
        return high_consequence, low_consequence, ineffective_evacuation, ac_greater_than_100, ac_greater_than_300
    
    
    def get_risk_curves(self, hist_dict):
        """
        Calculates risk curves using probability histograms.
        probability histograms signifies an AC, BU, POP histogram 
        where the initial probability is multiplied with the fire probability
        """
        output={}
        ac = hist_dict['AC']
        bu = hist_dict['BU']
        pop = hist_dict['POP']

        output['ac'] = ac
        output['pop'] = pop
        output['bu'] = bu
        safety_histogram = pop
        safety_curve = []

        if len(pop):
            sorted_keys = sorted(pop.keys())
            starting_sum = sum([pop[key] for key in sorted_keys]) # sum of all probabilities
            safety_curve.append([pop[sorted_keys[0]], starting_sum])
            for idx in range(1,len(sorted_keys)):
                prev_prob= pop[sorted_keys[idx-1]]
                current_num = sorted_keys[idx]
                starting_sum -= prev_prob
                safety_curve.append([current_num, starting_sum])

        output['safety'] = safety_curve
        output['safety_histogram'] = pop    
        # print(output['safety'])
        financial_bu = [[arr[0] * 10 ** 6, arr[1]] for arr in
                        bu.items() if arr[0] is not None]  # multiplies bu value by 10^6
        # print(bu)
        # print(financial_bu)
        financial_ac = [[arr[0] * 1175, arr[1]] for arr in# UPDATE
                        ac.items() if arr[0] is not None]  # multiplies ac value by 1175
        # print(financial_ac)
        merged_hist = {}
        for current_bu in financial_bu:
            if merged_hist.get(current_bu[0]):
                merged_hist[current_bu[0]].append(current_bu[1])
            else:
                merged_hist[current_bu[0]] = [current_bu[1]]
        for current_ac in financial_ac:
            if merged_hist.get(current_ac[0]):
                merged_hist[current_ac[0]].append(current_ac[1])
            else:
                merged_hist[current_ac[0]] = [current_ac[1]]
    #     print(merged_hist)
        ordered_hist = []
        for value in sorted(list(merged_hist)):
            ordered_hist.append([value, sum(merged_hist[value]) / 2])
    #         ordered_hist.append([value, reduce(lambda x, y: x * y, merged_hist[value]) / 2])
        financial_histogram = ordered_hist
        # print(ordered_hist)
        financial_curve = []
        if len(ordered_hist):
            starting_sum = sum([arr[1] for arr in ordered_hist]) # sum of all probabilities
            financial_curve.append([ordered_hist[0][0], starting_sum])
            # starting_sum = starting_sum - a[0][0]
            for idx in range(1,len(ordered_hist)):
                prev_num = ordered_hist[idx-1][1]
                current_num = ordered_hist[idx][0]
                starting_sum -= prev_num
                financial_curve.append([current_num,starting_sum])
    #         sorted_keys = sorted(pop.keys())
        output['financial_histogram'] = financial_histogram
        output['financial'] = financial_curve
        # print(financial_curve)
        
        probability_1 = sum([current_ac[1] for current_ac in ac.items() if current_ac[0] is not None and float(current_ac[0]) < 300])
        ac_over_300 = sum([current_ac[1] for current_ac in ac.items() if current_ac[0] is not None and float(current_ac[0]) > 300])
        bu_under_50 = sum([current_bu[1] for current_bu in bu.items() if current_bu[0] is not None and float(current_bu[0]) < 50])
        bu_over_50 = sum([current_bu[1] for current_bu in bu.items() if current_bu[0] is not None and float(current_bu[0]) > 50])

        probability_2 = ac_over_300 * bu_under_50
        probability_3 = ac_over_300 * bu_over_50
        reliability_curve = [
            [0.04, probability_3 + probability_2 + probability_1],
            [0.7, (probability_3 + probability_2)],
            [223, (probability_3)]
        ]
        # if incorrect numbers, same x values, y values prob1, prob2, prob 3
        reliability_histogram = [
            [0.04,  probability_1],
            [0.7,  (probability_2)],
            [223, (probability_3)]
        ]
        output['reliability_histogram'] = reliability_histogram
        output['reliability'] = reliability_curve
        return output
    
    
    def calculate_endstate(self,):
        pass
    
    
    def read_initial_settings(self,):
        configurations =  ['psps_not_performed','psps','ignition1','ignition2','ignition3','outage1','outage2','catastrophe_ac_threshold','catastrophe_bu_threshold','catastrophe_pop_threshold']
        config_str = '=============CONFIGURATIONS============\n\n'
        for config in configurations:
            value = self.__getattribute__(config)
            if value is None:
                config_str += '{}\t(NOT SET)\n'.format(config)
            else:
                config_str += '{}\t{}\n'.format(config, value)
        print(config_str, '\n=========================================\n')
    
    
    def run(self,):
        pass
    

In [12]:
class TransmissionQuantification(Quantification):
    def __init__(self, veg_df, struct_df, **kwargs):
        Quantification.__init__(self, **kwargs)
        # required inputs
        self.veg_df = veg_df
        self.struct_df = struct_df
        self.structure_ignition1 = kwargs.get('structure_ignition1', 0.01)
        self.structure_ignition3 = kwargs.get('structure_ignition3', 0.01)
        self.vegetation_ignition1 = kwargs.get('vegetation_ignition1', 0.01)
        self.vegetation_ignition3 = kwargs.get('vegetation_ignition3', 0.01)
        
        
        
        self.transmission_lines_common = set(veg_df['transmission_id'].values) & set(
            struct_df['transmission_id'].values)
        self.exclusive_veg_trans = [value for value in (set(
            veg_df['transmission_id'].values) ^ self.transmission_lines_common) if value == value]
        self.exclusive_struct_trans = set(
            struct_df['transmission_id'].values) ^ self.transmission_lines_common
        self.all_transmission_lines = [value for value in (set(
            self.exclusive_veg_trans) | self.exclusive_struct_trans | self.transmission_lines_common) if value == value]

        # attributes that will be populated once the 'run' method is invoked
        self.end_states_by_veg_and_struct = None
        self.hist_by_tl = None
        self.quantifications_by_asset = None
        self.latest_line_hist_reduced = None
        self.hist_by_tree_and_structure = None  # tl->tree->hist, tl->struct->hist
        self.prob_hist_by_tree_and_structure = None
        # totals
        self.transmisssion_level_endstates = None
        self.configurations = None
        self.total_fires = None
        self.risk_curves_by_line = None
        endstates_by_transmission_line = None
        self.total_fires_from_lines = None

    def get_hists(self, row, count_needed=True):
        """takes a df row and returns the hists, also keeps count of valid hists to later evenly divide"""
        ac = row.AC_2020
        pop = row.POP_2020
        bu = row.BU_2020
        ac_count = 0
        bu_count = 0
        pop_count = 0
        if ac != ac or 'nan' in ac:
            ac = '[]'
        else:
            ac_count += 1
        if bu != bu or 'nan' in bu:
            bu = '[]'
        else:
            bu_count += 1
        if pop != pop or 'nan' in pop:
            pop = '[]'
        else:
            pop_count += 1
        if count_needed:
            return {'AC': literal_eval(ac), 'BU': literal_eval(bu), 'POP': literal_eval(pop)}, [ac_count, bu_count, pop_count]
        else:
            return {'AC': literal_eval(ac), 'BU': literal_eval(bu), 'POP': literal_eval(pop)}

    def calculate_endstates(self,):
        # collects hists at line level with a hist count (ac_count, bu_count, pop_count)
        hist_by_tl = {}
        # collects quantifications values by transmission_id and structure/veg id
        quantifications_by_asset = {}
        hist_by_tree_and_structure = {}
        configurations = defaultdict(list)
        print('common')
        epss = 1
        for line in self.log_progress(self.transmission_lines_common):
            # for the current transmission line, creates a dictionary aggregate all
            # AC, BU, and POP distributions
            hist_by_tl[line] = defaultdict(list)
            ac_count, bu_count, pop_count = 0, 0, 0
            # gets all the structures that belong to the current line
            structures_by_line = self.struct_df[self.struct_df.transmission_id == line]
            # gets all the vegetation that belongs to the current line
            veg_by_line = self.veg_df[self.veg_df.transmission_id == line]
            # dictionary created to store histograms 
            hist_by_tree_and_structure[line] = {}
            # dictionary created to store quantifications for this line 
            quantifications_by_asset[line] = {}
            # will collection multiple probabilities at for repeated values
            grouped_hist = {'AC': defaultdict(list), 'BU': defaultdict(
                list), 'POP': defaultdict(list)}
            # iterates through all of the structures that belong to this line
            for row in structures_by_line.itertuples():
                self.ignition1 = self.structure_ignition1
                self.ignition3 = self.structure_ignition3
                struct_id = row.structure_id
                psps = row.psps if self.psps is None else self.psps
                outage_1 = self.get_outage(
                    row.outage_1, 0) if self.outage1 is None else self.outage1
                outage_2 = self.get_outage(
                    row.outage_2, 0) if self.outage2 is None else self.outage2
                # gets the histogram for this structure and gets a count of 1 for each AC, BU, POP is they exist, else 0
                hist, counts = self.get_hists(row)
                # stores histogram by transmission line and structure id
                hist_by_tree_and_structure[line][struct_id] = hist
                # increment by the count
                ac_count += counts[0]
                bu_count += counts[1]
                pop_count += counts[2]
                # Gets consquences based on histogram and aggregates grouped_hist
                high_consequence, low_consequence, ineffective_evacuation, ac_greater_than_100,  ac_greater_than_300 = self.get_consequences(
                    hist, grouped_hist)
                configurations[line].append({
                    'id': struct_id,
                    'psps': psps, 'outage1': outage_1, 'outage2': outage_2,
                    'high_consequence': high_consequence, 'low_consequence': low_consequence,
                    'ineffective_evacuation': ineffective_evacuation,
                })
                # stores quantification at the line/structure level
                quantifications_by_asset[line][struct_id] = self.get_quanitification(
                    high_consequence, psps, ineffective_evacuation, outage_1, outage_2, low_consequence, ac_greater_than_100, ac_greater_than_300, epss)

            for row in veg_by_line.itertuples():
                self.ignition1 = self.vegetation_ignition1
                self.ignition3 = self.vegetation_ignition3
                veg_id = row.TREEID
                psps = row.psps if self.psps is None else self.psps
                outage_1 = self.get_outage(
                    row.outage_1, 0) if self.outage1 is None else self.outage1
                outage_2 = self.get_outage(
                    row.outage_2, 0) if self.outage2 is None else self.outage2
                hist, counts = self.get_hists(row)
                hist_by_tree_and_structure[line][veg_id] = hist
                ac_count += counts[0]
                bu_count += counts[1]
                pop_count += counts[2]
                high_consequence, low_consequence, ineffective_evacuation, ac_greater_than_100,  ac_greater_than_300 = self.get_consequences(
                    hist, grouped_hist)

                configurations[line].append({
                    'id': veg_id, 'psps': psps, 'outage1': outage_1, 'outage2': outage_2,
                    'high_consequence': high_consequence, 'low_consequence': low_consequence,
                    'ineffective_evacuation': ineffective_evacuation
                })
                quantifications_by_asset[line][veg_id] = self.get_quanitification(
                    high_consequence, psps, ineffective_evacuation, outage_1, outage_2, low_consequence, ac_greater_than_100, ac_greater_than_300, epss)
            
            # once all vegetation and structure at this level
            # (these iterations consists of tranmission lines that have both vegetation and structures)
            hist_by_tl[line] = {**grouped_hist, 'ac_count': ac_count,
                                'bu_count': bu_count, 'pop_count': pop_count}

        print('vegetation')
        for line in self.log_progress(self.exclusive_veg_trans):
            hist_by_tl[line] = defaultdict(list)
            veg_by_line = self.veg_df[self.veg_df.transmission_id == line]
            quantifications_by_asset[line] = {}
            grouped_hist = {'AC': defaultdict(list), 'BU': defaultdict(
                list), 'POP': defaultdict(list)}
            ac_count, bu_count, pop_count = 0, 0, 0
            hist_by_tree_and_structure[line] = {}
            for row in veg_by_line.itertuples():
                self.ignition1 = self.vegetation_ignition1
                self.ignition3 = self.vegetation_ignition3
                veg_id = row.TREEID
                psps = row.psps if self.psps is None else self.psps
                outage_1 = self.get_outage(
                    row.outage_1, 0) if self.outage1 is None else self.outage1
                outage_2 = self.get_outage(
                    row.outage_2, 0) if self.outage2 is None else self.outage2
                hist, counts = self.get_hists(row)
                hist_by_tree_and_structure[line][veg_id] = hist
                ac_count += counts[0]
                bu_count += counts[1]
                pop_count += counts[2]
                high_consequence, low_consequence, ineffective_evacuation, ac_greater_than_100,  ac_greater_than_300 = self.get_consequences(
                    hist, grouped_hist)
                configurations[line].append({
                    'id': veg_id,
                    'psps': psps, 'outage1': outage_1, 'outage2': outage_2,
                    'high_consequence': high_consequence, 'low_consequence': low_consequence,
                    'ineffective_evacuation': ineffective_evacuation
                })
                quantifications_by_asset[line][veg_id] = self.get_quanitification(
                    high_consequence, psps, ineffective_evacuation, outage_1, outage_2, low_consequence, ac_greater_than_100, ac_greater_than_300, epss)
            hist_by_tl[line] = {**grouped_hist, 'ac_count': ac_count,
                                'bu_count': bu_count, 'pop_count': pop_count}

        print('structures')
        for line in self.log_progress(self.exclusive_struct_trans):
            hist_by_tl[line] = defaultdict(list)
            structures_by_line = self.struct_df[self.struct_df.transmission_id == line]
            quantifications_by_asset[line] = {}
            grouped_hist = {'AC': defaultdict(list), 'BU': defaultdict(
                list), 'POP': defaultdict(list)}
            ac_count, bu_count, pop_count = 0, 0, 0
            hist_by_tree_and_structure[line] = {}
            for row in structures_by_line.itertuples():
                self.ignition1 = self.structure_ignition1
                self.ignition3 = self.structure_ignition3
                struct_id = row.structure_id
                psps = row.psps if self.psps is None else self.psps
                outage_1 = self.get_outage(
                    row.outage_1, 0) if self.outage1 is None else self.outage1
                outage_2 = self.get_outage(
                    row.outage_2, 0) if self.outage2 is None else self.outage2
                hist, counts = self.get_hists(row)
                hist_by_tree_and_structure[line][struct_id] = hist
                ac_count += counts[0]
                bu_count += counts[1]
                pop_count += counts[2]
                high_consequence, low_consequence, ineffective_evacuation, ac_greater_than_100,  ac_greater_than_300 = self.get_consequences(
                    hist, grouped_hist)
                configurations[line].append({
                    'id': struct_id,
                    'psps': psps, 'outage1': outage_1, 'outage2': outage_2,
                    'high_consequence': high_consequence, 'low_consequence': low_consequence,
                    'ineffective_evacuation': ineffective_evacuation,
                })
                quantifications_by_asset[line][struct_id] = self.get_quanitification(
                    high_consequence, psps, ineffective_evacuation, outage_1, outage_2, low_consequence, ac_greater_than_100, ac_greater_than_300, epss)
            hist_by_tl[line] = {**grouped_hist, 'ac_count': ac_count,
                                'bu_count': bu_count, 'pop_count': pop_count}

        self.hist_by_tl = hist_by_tl
        self.quantifications_by_asset = quantifications_by_asset
        self.hist_by_tree_and_structure = hist_by_tree_and_structure
        self.configurations = configurations

    """
    Creates probability histograms at the Vegetation and Structure level
    Creates probability histograms at the Transmission Level from aggregating and dividing by the number of histograms (ac_count)
    """

    def get_line_level_reduced(self,):
        """
        Generates line level probability histograms using structure and tree histograms
        """
        prob_hist_by_tree_and_structure = []
        line_level_prob_histograms = {}
        # iterates through all lines and assets (tree and structures)
        # in order to get histograms
        for line, assets in self.log_progress(self.hist_by_tree_and_structure.items()):
            # for every line we create an aggregation dictionary
            transmission_prob_hist_aggregation = {'AC': defaultdict(
                list), 'BU': defaultdict(list), 'POP': defaultdict(list)}
            # iterating through all the histograms stored at the tree/structure level
            for asset_id, hist in assets.items():  # where asset_id is tree/structure id
                # getting the fire probability
                fire_prob = (sum(self.quantifications_by_asset[line][asset_id].values(
                )) - self.quantifications_by_asset[line][asset_id]['safe_probability'])
                # calculating probability histograms using the current histogram and fire probability
                prob_hist = self.get_prob_hist(
                    fire_prob, hist, transmission_prob_hist_aggregation
                )
                # Structure/Vegetation level risk curve
                risk_curve = self.get_risk_curves(prob_hist)
                # stores risk curve and prob hist into prob_hist_by_tree_and_structure
                prob_hist_by_tree_and_structure.append(
                    {'id': asset_id,
                     **prob_hist,
                     **risk_curve
                     })
            # once every veg and struct have iterated store prob hist aggregation here, along with ac,bu,pop count
            line_level_prob_histograms[line] = {**transmission_prob_hist_aggregation,
                                                'ac_count': self.hist_by_tl[line]['ac_count'],
                                                'pop_count': self.hist_by_tl[line]['pop_count'],
                                                'bu_count': self.hist_by_tl[line]['bu_count']
                                                }
        # once all the lines are iterated through, **attr** prob_hist_by_tree_and_structure will be set to prob_hist_by_tree_and_structure
        self.prob_hist_by_tree_and_structure = prob_hist_by_tree_and_structure

        # creates probability histograms at the transmission line level using aggregation of line_level_prob_histograms
        # also uses the AC, BU, and POP count
        latest_line_hist_reduced = {}
        for line_id, hist in self.log_progress(line_level_prob_histograms.items()):
            latest_line_hist_reduced[line_id] = {
                'AC': defaultdict(int),
                'BU': defaultdict(int),
                'POP': defaultdict(int)
            }
            for _type, pairs in hist.items():
                if _type == 'AC':
                    latest_line_hist_reduced[line_id]['AC'] = {
                        value: sum(probs)/hist['ac_count']
                        for value, probs in pairs.items() if hist['ac_count'] != 0
                    }
                if _type == 'POP':
                    latest_line_hist_reduced[line_id]['POP'] = {
                        value: sum(probs)/hist['pop_count']
                        for value, probs in pairs.items() if hist['bu_count'] != 0}
                if _type == 'BU':
                    latest_line_hist_reduced[line_id]['BU'] = {
                        value: sum(probs)/hist['bu_count']
                        for value, probs in pairs.items() if hist['pop_count'] != 0}
        self.latest_line_hist_reduced = latest_line_hist_reduced

    def get_line_level_risk_curves(self,):
        """
        Generates risk curves using aggregate histograms at the line level, and stores the results into a list
        """
        risk_curves_by_line = []
        # using line level aggregation of histogram, line level risk curves are calculated
        for line_id, prob_hist in self.latest_line_hist_reduced.items():
            if line_id == line_id:
                risk_curve = self.get_risk_curves(prob_hist)
                risk_curves_by_line.append({
                    'id': str(int(line_id)),
                    'financial': risk_curve['financial'],
                    'reliability': risk_curve['reliability'],
                    'safety': risk_curve['safety'],
                    'pop': risk_curve['pop'],
                    'financial_histogram': risk_curve['financial_histogram'],
                    'reliability_histogram': risk_curve['reliability_histogram']
                })
            else:
                print(line_id)
        self.risk_curves_by_line = risk_curves_by_line

    def calculate_ac_bu_pop_and_risk_totals(self,):
        """
        Calculates totals for AC, BU, POP, and risk curves from trees and structures
        """
        ac_total = 0
        pop_total = 0
        bu_total = 0
        financial_total = 0
        reliability_total = 0
        safety_total = 0
        # calculates total for above at the structure and tree level
        for prob_hist in self.log_progress(self.prob_hist_by_tree_and_structure):
            for value, prob in prob_hist['AC'].items():
                ac_total += value * prob
            for value, prob in prob_hist['POP'].items():
                pop_total += value * prob
                safety_total += value/2000 * prob
            for value, prob in prob_hist['BU'].items():
                bu_total += value * prob
            for arr in prob_hist['financial_histogram']:
                financial_total += arr[0] * arr[1]
            for arr in prob_hist['reliability_histogram']:
                reliability_total += arr[0] * arr[1]

        self.ac_total = ac_total
        self.pop_total = pop_total
        self.bu_total = bu_total
        self.financial_total = financial_total
        self.reliability_total = reliability_total
        self.safety_total = safety_total


    def get_endstate_description(self,):
        
        """
        Generates a stats table using endstates at Structure and Vegetation level
        """
        endstate_dict = defaultdict(list)
        for _, assets in self.quantifications_by_asset.items():
            for _, endstates in assets.items():
                for endstate, value in endstates.items():
                    endstate_dict[endstate].append(value)
        return pd.DataFrame(endstate_dict).describe()


    def get_transmission_level_endstate(self,):
        """
        Calculates endstates at transmission system level using probabilities at Structure and Vegetation level
        """
        endstate_dict = defaultdict(list)
        for _, assets in self.quantifications_by_asset.items():
            for _, endstates in assets.items():
                for endstate, value in endstates.items():
                    endstate_dict[endstate].append(value)

        transmission_level_endstates = {
            'catastrophic_fire': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['catastrophic_fire']]),
            'destructive_fire': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['destructive_fire']]),
            'fire_probability': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['fire_probability']]),
            'fire_probability_no_injury': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['fire_probability_no_injury']])
        }
        transmission_level_endstates['safety_probability'] = reduce(
            lambda x, y: x*y, [1-value for value in transmission_level_endstates.values()])
        self.transmisssion_level_endstates = transmission_level_endstates


    def calculate_total_fires(self, ):
        """
        Calculates total fires by adding all probabilities together at the structure and tree level
        """
        fires_by_endstates_trans = defaultdict(int)
        for _, assets in self.quantifications_by_asset.items():
            for _, endstates in assets.items():
                for endstate, value in endstates.items():
                    fires_by_endstates_trans[f'{endstate} (total fires)'] += (value)
        self.total_fires =  fires_by_endstates_trans


    def calculate_total_fires_from_lines(self, ):
        """
        Calculates total fires by adding all probabilities together at the transmission line level
        """
        fires_by_endstates_trans_lines = defaultdict(int)
        for _, endstates in self.endstates_by_transmission_line.items():
            for endstate, value in endstates.items():
                fires_by_endstates_trans_lines[f'{endstate} (total fires)'] += (value)
        self.total_fires_from_lines = fires_by_endstates_trans_lines


    
    def calculate_transmission_line_level_endstates(self,):
        """
        Using aggregation, calculates endstates from the structure / tree level to transmission level
        """
        endstates_by_transmission_line_list = {}
        for transmission_id, assets in self.quantifications_by_asset.items():
            endstates_by_transmission_line_list[transmission_id] = defaultdict(list)
            for _, endstates in assets.items():
                for endstate, value in endstates.items():
                    endstates_by_transmission_line_list[transmission_id][endstate].append(1 - value)
        
        endstates_by_transmission_line = {}
        for transmission_id, endstates in endstates_by_transmission_line_list.items():
            endstates_by_transmission_line[transmission_id] = {}
            for endstate, values in endstates.items():
                endstates_by_transmission_line[transmission_id][endstate] = 1 - np.prod(values)
        
        self.endstates_by_transmission_line = endstates_by_transmission_line
        
        
    def totals(self, include_configuration=False, from_lines=True) -> pd.DataFrame:
        """
        displays total as a DF
        """
        if from_lines:
            fires_dict = { key: [value] for key,value in 
               self.total_fires_from_lines.items()
            }
        else:
            fires_dict = { key: [value] for key,value in 
               self.total_fires.items()
            }
        totals_dict = {
            'AC_TOTAL':[self.ac_total],
            'BU_TOTAL': [self.bu_total],
            'POP_TOTAL': [self.pop_total],
            'Financial_Total': [self.financial_total],
            'Reliability_Total': [self.reliability_total],
            'Safety_Total': [self.safety_total],
            **{ key: [value] for key,value in 
                self.transmisssion_level_endstates.items()
            },
            **fires_dict
        }
        if include_configuration:
            totals_dict['psps_not_performed'] = self.psps_not_performed
            totals_dict['psps'] = self.psps
            totals_dict['ignition1'] = self.ignition1
            totals_dict['ignition2'] = self.ignition2
            totals_dict['ignition3'] = self.ignition3
            totals_dict['outage1'] = self.outage1
            totals_dict['outage2'] = self.outage2
            totals_dict['catastrophe_ac_threshold'] = self.catastrophe_ac_threshold
            totals_dict['catastrophe_bu_threshold'] = self.catastrophe_bu_threshold
            totals_dict['catastrophe_pop_threshold'] = self.catastrophe_pop_threshold
        return pd.DataFrame(totals_dict)
    

    def read_initial_settings(self,):
        configurations =  ['psps_not_performed','psps','ignition1',
                           'ignition2','ignition3','outage1','outage2',
                           'catastrophe_ac_threshold','catastrophe_bu_threshold','catastrophe_pop_threshold',
                           'vegetation_ignition1', 'vegetation_ignition3','structure_ignition1', 'structure_ignition3', 
                          ]
        config_str = '=============CONFIGURATIONS============\n\n'
        for config in configurations:
            value = self.__getattribute__(config)
            if value is None:
                config_str += '{}\t(NOT SET)\n'.format(config)
            else:
                config_str += '{}\t{}\n'.format(config, value)
        print(config_str, '\n=========================================\n')
                    
    def run(self,):
        self.read_initial_settings()
        print('calculating endstates...')
        print('\t\t\t\t\t\t\t\t\t[#    ]')
        self.calculate_endstates()
        print('endstates complete, now calculating prob histograms for line and asset/tree')
        print('\t\t\t\t\t\t\t\t\t[##   ]')
        self.get_line_level_reduced()
        print('getting ac bu pop and risktotals')
        print('\t\t\t\t\t\t\t\t\t[#### ]')
        self.calculate_ac_bu_pop_and_risk_totals()
        print('calculating risk curves by line')
        print('\t\t\t\t\t\t\t\t\t[#### ]')
        self.get_line_level_risk_curves()
        self.get_transmission_level_endstate()
        self.calculate_total_fires()
        self.calculate_transmission_line_level_endstates()
        self.calculate_total_fires_from_lines()
        print('\t\t\t\t\t\t\t\t\t[#####]')
        print('COMPLETE')


In [5]:
class DistributionQuantification(Quantification):
    def __init__(self, dist_df, **kwargs):
        Quantification.__init__(self, **kwargs)
        self.dist_df = dist_df
        self.quantifications = {}
        self.distribution_level_endstates = None
        self.prob_hist_by_line = None
        self.risk_curves_by_line = None
        self.config_by_line = None
        self.mitigation_endstates = {
            'evm': {},
            'hardening': {},
            'combined': {},
            'underground': {},
        }
        self.total_fires = None

    def get_hists(self, row):
        """takes a df row and returns the hists"""
        ac = row['AC_2020']
        pop = row['POP_2020']
        bu = row['BU_2020']
        if ac != ac or 'nan' in ac:
            ac = '[]'
        if bu != bu or 'nan' in bu:
            bu = '[]'
        if pop != pop or 'nan' in pop:
            pop = '[]'
        return {'AC': literal_eval(ac), 'BU': literal_eval(bu), 'POP': literal_eval(pop)}

    """
    Gets columns from get_outage_inputs 11 cols
    """

    def get_outage(self, **args):
        # 1-((1-p1)*…*(1-p10))
        """
        Calculates outage
        """
        start = 1
        for arg, value in args.items():
            start *= 1 - value
        return 1 - start


    def get_outage_inputs(self, row_dict):
        required_cols = [
            'weighted_support_structure_equipment_cause_probability_ignition_total',
            'weighted_support_structure_equipment_electrical_probability_ignition_total',
            'weighted_transformer_equipment_cause_probability_ignition_total',
            'weighted_transformer_equipment_leaking_probability_ignition_total',
            'weighted_animal_bird_probability_ignition_total',
            'weighted_animal_other_probability_ignition_total',
            'weighted_animal_squirrel_probability_ignition_total',
            'weighted_other_equipment_type_probability_ignition_total',
            'weighted_primary_conductor_probability_ignition_total',
            'weighted_secondary_conductor_probability_ignition_total',
            'weighted_third_party_balloon_probability_ignition_total',
            'weighted_third_party_other_probability_ignition_total',
            'weighted_third_party_vehicle_probability_ignition_total',
            'weighted_vegetation_branch_probability_ignition_total',
            'weighted_vegetation_other_probability_ignition_total',
            'weighted_vegetation_trunk_probability_ignition_total',
            'weighted_voltage_control_equipment_type_probability_ignition_total'
        ]
    
        return {col: row_dict[col] for col in required_cols}

    def calculate_endstate(self,):
        """
        Calculates endstate for CPZs
        """
        quantifications = {}
        config_by_line = {}
        # iterates through all of the cpzs
        for idx in range(len(self.dist_df)):
            row = self.dist_df.iloc[idx]
            # dictionary to store quantifications by cpz_is
            quantifications[row.circuit_protection_zone_id] = {}
            histograms = self.get_hists(row)
            outage1 = 1 if self.outage1 is None else self.outage1
            # calculates outage2
            outage2 = self.get_outage(
                **self.get_outage_inputs(row)) if self.outage2 is None else self.outage2
            high_consequence, low_consequence, ineffective_evacuation, ac_greater_than_100, ac_greater_than_300 = self.get_consequences(
                histograms
                )
            psps = row.psps if self.psps is None else self.psps
            # store configuration at the cpz level
            config_by_line[row.circuit_protection_zone_id] = {
                'psps': psps, 'high_consequence': high_consequence, 'low_consequence': low_consequence,
                'ineffective_evacuation': ineffective_evacuation, 'outage2': outage2, 'outage1': outage1
            }
            # quantification is calculated and stored
            quantifications[row.circuit_protection_zone_id] = self.get_quanitification(
                high_consequence,
                psps,
                ineffective_evacuation,
                outage1,
                outage2,
                low_consequence,
                ac_greater_than_100,
                ac_greater_than_300,
                row.epss
            )
        self.quantifications = quantifications
        self.config_by_line = config_by_line

    def get_distribution_level_endstate(self,):
        # will aggregate all quantifications from the CPZ level to the distibution
        endstate_dict = defaultdict(list)
        for _, endstates in self.quantifications.items():
            for endstate, value in endstates.items():
                endstate_dict[endstate].append(value)

        distribution_level_endstates = {
            'catastrophic_fire': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['catastrophic_fire']]),
            'destructive_fire': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['destructive_fire']]),
            'fire_probability': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['fire_probability']]),
            'fire_probability_no_injury': 1 - reduce(lambda x, y: x*y, [1 - value for value in endstate_dict['fire_probability_no_injury']])
        }
        distribution_level_endstates['safety_probability'] = reduce(
            lambda x, y: x*y, [1-value for value in distribution_level_endstates.values()])
        self.distribution_level_endstates = distribution_level_endstates

    def get_endstate_description(self,):
        endstate_dict = defaultdict(list)
        for _, endstates in self.quantifications.items():
            for endstate, value in endstates.items():
                endstate_dict[endstate].append(value)
        return pd.DataFrame(endstate_dict).describe()

    def calculate_ac_bu_pop_and_risk_totals(self,):
        """
        Calculates totals for risk curves and AC, BU, and POP
        """
        ac_total = 0
        pop_total = 0
        bu_total = 0
        financial_total = 0
        reliability_total = 0
        safety_total = 0
        for pz in self.log_progress(self.prob_hist_by_line):
            prob_hist = self.prob_hist_by_line[pz]
            for value, prob in prob_hist['AC'].items():
                ac_total += value * prob
            for value, prob in prob_hist['POP'].items():
                pop_total += value * prob
                safety_total += value/2000 * prob
            for value, prob in prob_hist['BU'].items():
                bu_total += value * prob
            prob_hist = self.risk_curves_by_line[pz]
            for arr in prob_hist['financial_histogram']:
                financial_total += arr[0] * arr[1]
            for arr in prob_hist['reliability_histogram']:
                reliability_total += arr[0] * arr[1]

        self.ac_total = ac_total
        self.pop_total = pop_total
        self.bu_total = bu_total
        self.financial_total = financial_total
        self.reliability_total = reliability_total
        self.safety_total = safety_total

    def calculate_total_fires(self, ):
        fires_by_endstates_dist = defaultdict(int)
        for _, endstates in self.quantifications.items():
            for endstate, value in endstates.items():
                fires_by_endstates_dist[f'{endstate} (total fires)'] += value
        self.total_fires = fires_by_endstates_dist

    def get_prob_hist_and_curves(self,):
        """
        Calculates probability histograms and risk curves at the line level using quantification at the CPZ level
        """
        histograms_by_pz = {}
        risk_curves_by_pz = {}
        # iterates through all quantifications 
        for pz in self.quantifications:
            fire_prob = 1 - self.quantifications[pz]['safe_probability']
            row = self.dist_df[self.dist_df.circuit_protection_zone_id == pz].iloc[0]
            histograms = self.get_hists(row)
            hist = self.get_prob_hist(fire_prob, histograms)
            histograms_by_pz[pz] = hist
            risk_curves_by_pz[pz] = {'id': pz, **self.get_risk_curves(hist)}
        self.prob_hist_by_line = histograms_by_pz
        self.risk_curves_by_line = risk_curves_by_pz
            
        
    def totals(self, include_configuration=False) -> pd.DataFrame:
        """
        Displays totals for risk curves and AC, BU, and POP
        """
        totals_dict = {
            'AC_TOTAL':[self.ac_total],
            'BU_TOTAL': [self.bu_total],
            'POP_TOTAL': [self.pop_total],
            'Financial_Total': [self.financial_total],
            'Reliability_Total': [self.reliability_total],
            'Safety_Total': [self.safety_total],
            **{ key: [value] for key,value in 
                self.distribution_level_endstates.items()
            },
            **{ key: [value] for key,value in 
               self.total_fires.items()
            }
        }
        if include_configuration:
            totals_dict['psps_not_performed'] = self.psps_not_performed
            totals_dict['psps'] = self.psps
            totals_dict['ignition1'] = self.ignition1
            totals_dict['ignition2'] = self.ignition2
            totals_dict['ignition3'] = self.ignition3
            totals_dict['outage1'] = self.outage1
            totals_dict['outage2'] = self.outage2
            totals_dict['catastrophe_ac_threshold'] = self.catastrophe_ac_threshold
            totals_dict['catastrophe_bu_threshold'] = self.catastrophe_bu_threshold
            totals_dict['catastrophe_pop_threshold'] = self.catastrophe_pop_threshold
            totals_dict['EPSS MEAN'] = self.dist_df.epss.mean()
        return pd.DataFrame(totals_dict)
    
    
    def read_initial_settings(self, ):
        print(f'EPSS AVERAGE \t\t\t {self.dist_df.epss.mean()}')
        super().read_initial_settings()
        

    def run(self,):
        self.read_initial_settings()
        self.calculate_endstate()
        self.get_distribution_level_endstate()
        self.get_prob_hist_and_curves()
        self.calculate_ac_bu_pop_and_risk_totals()
        self.calculate_total_fires()


In [ ]:
dist = DistributionQuantification(
    dist_df=distribution
)
dist.run()

## Distribution Totals

In [9]:
dist.totals(include_configuration=True).transpose().to_dict()[0]

{'AC_TOTAL': 337227.2337250204,
 'BU_TOTAL': 5133.530980055523,
 'POP_TOTAL': 8658.568565708174,
 'Financial_Total': 2764886489.841276,
 'Reliability_Total': 447.06805312991077,
 'Safety_Total': 4.3292842828540286,
 'catastrophic_fire': 1.5085173998841128e-08,
 'destructive_fire': 0.057775663755295414,
 'fire_probability': 8.159513880712232e-05,
 'fire_probability_no_injury': 1.0,
 'safety_probability': 0.0,
 'catastrophic_fire (total fires)': 1.508517409388938e-08,
 'destructive_fire (total fires)': 0.05950896804418474,
 'fire_probability (total fires)': 8.159663017044348e-05,
 'fire_probability_no_injury (total fires)': 540.8970083751619,
 'safe_probability (total fires)': 11308.043401045068,
 'ac_greater_than_100_fire (total fires)': 12.616972269967107,
 'ac_greater_than_300_fire (total fires)': 10.681702743686468,
 'psps_not_performed': 0.05,
 'psps': None,
 'ignition1': 1,
 'ignition2': 0,
 'ignition3': 1,
 'outage1': None,
 'outage2': None,
 'catastrophe_ac_threshold': 300,
 'cat

In [10]:
trans = TransmissionQuantification(
    veg_df=vegetation,
    struct_df=structures,
)
trans.run()

=============CONFIGURATIONS============

psps_not_performed	0.05
psps	(NOT SET)
ignition1	1
ignition2	0
ignition3	1
outage1	(NOT SET)
outage2	(NOT SET)
catastrophe_ac_threshold	300
catastrophe_bu_threshold	50
catastrophe_pop_threshold	1
vegetation_ignition1	0.01
vegetation_ignition3	0.01
structure_ignition1	0.01
structure_ignition3	0.01
 

calculating endstates...
									[#    ]
common


vegetation


structures


endstates complete, now calculating prob histograms for line and asset/tree
									[##   ]


getting ac bu pop and risktotals
									[#### ]


calculating risk curves by line
									[#### ]
									[#####]
COMPLETE


## Transmission Totals

In [11]:
trans.totals(from_lines=True).transpose().to_dict()[0]

{'AC_TOTAL': 2174.2028834442226,
 'BU_TOTAL': 31.327784920480383,
 'POP_TOTAL': 28.927745022548866,
 'Financial_Total': 16941236.65426401,
 'Reliability_Total': 0.19600492031829733,
 'Safety_Total': 0.014463872511274439,
 'catastrophic_fire': 8.659739592076221e-15,
 'destructive_fire': 0.0003292669992291941,
 'fire_probability': 8.188005828912992e-12,
 'fire_probability_no_injury': 0.8508023261312602,
 'safety_probability': 0.14914854799715058,
 'catastrophic_fire (total fires)': 8.659739592076221e-15,
 'destructive_fire (total fires)': 0.0003292743367047102,
 'fire_probability (total fires)': 8.188005828912992e-12,
 'fire_probability_no_injury (total fires)': 1.4121350076561776,
 'safe_probability (total fires)': 1355.9998951584807,
 'ac_greater_than_100_fire (total fires)': 0.08725211517044662,
 'ac_greater_than_300_fire (total fires)': 0.07453946399417133}